In [0]:
from google.colab import auth
auth.authenticate_user()


from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Embedding
from keras.layers import LSTM
from keras.layers import Conv1D,MaxPooling1D,Conv2D,MaxPooling2D,GlobalMaxPooling2D,Bidirectional,Reshape,Dropout
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
np.random.seed(3)


In [0]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
np.random.seed(3)
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/trainingSet',
                                                 target_size = (560, 420),
                                                 batch_size = 12,                                                
                                                 class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory('/content/gdrive/My Drive/Colab Notebooks/valve_id_00_+6db/testSet',
                                            target_size = (560, 420),
                                            batch_size = 8,                                            
                                            class_mode = 'categorical')

Found 120 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [0]:
model = Sequential()
model.add(Conv2D(28, kernel_size=(2, 2),
                 activation='relu',input_shape=(560,420,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(15, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(1, kernel_size=(1,1),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Reshape((-1,1)))
model.add(LSTM(200, activation='relu',return_sequences=True))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax' ))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 559, 419, 28)      364       
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 279, 209, 28)      0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 279, 209, 28)      0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 279, 209, 15)      435       
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 139, 104, 15)      0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 139, 104, 1)       16        
_________________________________________________________________
max_pooling2d_76 (MaxPooling (None, 69, 52, 1)       

In [0]:
early_stopping_callback = EarlyStopping(monitor='val_loss',mode='min', patience=5)
checkpoint_callback = ModelCheckpoint('best_model.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=10,
        validation_data=test_generator,
        validation_steps=10,
        callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/10
10/10 [==============================] - 57s 6s/step - loss: 0.6791 - acc: 0.6417 - val_loss: 0.5331 - val_acc: 0.8500

Epoch 00001: val_loss improved from inf to 0.53314, saving model to best_model.h5
Epoch 2/10
10/10 [==============================] - 52s 5s/step - loss: 0.4093 - acc: 0.8417 - val_loss: 0.2626 - val_acc: 0.9000

Epoch 00002: val_loss improved from 0.53314 to 0.26256, saving model to best_model.h5
Epoch 3/10
10/10 [==============================] - 51s 5s/step - loss: 0.3793 - acc: 0.8417 - val_loss: 0.1259 - val_acc: 0.9625

Epoch 00003: val_loss improved from 0.26256 to 0.12592, saving model to best_model.h5
Epoch 4/10
10/10 [==============================] - 52s 5s/step - loss: 0.3384 - acc: 0.8417 - val_loss: 0.6931 - val_acc: 0.5000

Epoch 00004: val_loss did not improve from 0.12592
Epoch 5/10
10/10 [==============================] - 52s 5s/step - loss: 0.3021 - acc: 0.9000 - val_loss: 0.2020 - val_acc: 0.9375

Epoch 00005: val_loss did not improve fr

In [0]:
print("--TrianSet_Evaluate--")
scores=model.evaluate_generator(train_generator,steps=12)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TrianSet_Evaluate--
acc: 94.44%


In [0]:
print("--TestSet_Evaluate--")

scores=model.evaluate_generator(test_generator,steps=8)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

--TestSet_Evaluate--
acc: 85.94%
